In [243]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, lxml
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [244]:
!pip install folium

In [245]:
import folium

In [246]:
wikiurl="https://es.wikipedia.org/wiki/Distrito_Nacional"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [247]:
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})

In [248]:
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
print(df.head())

                Sectores  Población
0  24 de abril            53870    
1  27 de febrero          20000    
2  30 de mayo             5904     
3  Altos de Arroyo Hondo  19617    
4  Arroyo Manzano         5894     


In [249]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='91LCVJBW8OyktK9Obt9l2hg1qxmaOvQHeErVDi4Nwzhs',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'holaproyectofinaldelcurso-donotdelete-pr-5yqimcvcp9bjmq'
object_key = 'cod-postal-sector.xlsx'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']

df_data_0 = pd.read_excel(body.read())
df_data_0.head()


,Sectores,cod_postal
0,Cristo Rey,10501.0
1,Domingo Savio,10304.0
2,Ensanche Capotillo,10407.0
3,Nuevo Arroyo Hondo,10507.0
4,Palma Real,10702.0


In [250]:
df_data_0["cod_postal"] = df_data_0["cod_postal"].fillna(99999).astype(int)

In [251]:
df_data_0.head()

,Sectores,cod_postal
0,Cristo Rey,10501
1,Domingo Savio,10304
2,Ensanche Capotillo,10407
3,Nuevo Arroyo Hondo,10507
4,Palma Real,10702


In [252]:
import requests

url = "https://api.foursquare.com/v3/places/search?ll=18.483402,-69.929611&radius=100000&limit=50"


headers = {
    "Accept": "application/json",
    "Authorization": "fsq3hXPPreyOaQh8ex2OiRY6yZGx5X6IqUFrwr4lhRybqDs="
}

response = requests.get(url, headers=headers).json()


print(response)

{'results': [{'fsq_id': '5481057d498e7d626697ed11', 'categories': [{'id': 13236, 'name': 'Italian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}}], 'chains': [], 'distance': 430, 'geocodes': {'main': {'latitude': 18.480267, 'longitude': -69.932012}}, 'link': '/v3/places/5481057d498e7d626697ed11', 'location': {'address': 'Jose Amado Soler', 'country': 'DO', 'cross_street': 'Pablito Casals', 'formatted_address': 'Jose Amado Soler (Pablito Casals), 10122 Santo Domingo', 'locality': 'Santo Domingo', 'postcode': '10122', 'region': 'Distrito Nacional'}, 'name': 'Bottega Fratelli', 'related_places': {}, 'timezone': 'America/Santo_Domingo'}, {'fsq_id': '4ca6356a14c33704ea61bd3b', 'categories': [{'id': 13002, 'name': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}}], 'chains': [], 'distance': 702, 'geocodes': {'main': {'latitude': 18.47885, 'longitude': -69.934236}}, 'link': '/v3/pla

In [253]:
venues = response['results']
venues_df = pd.json_normalize(response,"results")
venues_df.head(2)

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region,related_places.children,related_places.parent.fsq_id,related_places.parent.name
0,5481057d498e7d626697ed11,"[{'id': 13236, 'name': 'Italian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}}]",[],430,/v3/places/5481057d498e7d626697ed11,Bottega Fratelli,America/Santo_Domingo,18.480267,-69.932012,Jose Amado Soler,DO,Pablito Casals,"Jose Amado Soler (Pablito Casals), 10122 Santo Domingo",Santo Domingo,10122,Distrito Nacional,NaN,NaN,NaN
1,4ca6356a14c33704ea61bd3b,"[{'id': 13002, 'name': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}}]",[],702,/v3/places/4ca6356a14c33704ea61bd3b,Delacasa Pastelería,America/Santo_Domingo,18.478850,-69.934236,Agustin Lara No. 41,DO,,"Agustin Lara No. 41, 10122 Santo Domingo",Santo Domingo,10122,Distrito Nacional,NaN,NaN,NaN


In [254]:
cols = ['name', 'categories', 'geocodes.main.latitude', 'geocodes.main.longitude','location.postcode']
venues_df = venues_df.loc[:, cols]

pd.set_option("display.max_colwidth" , -1)
venues_df.head()

,name,categories,geocodes.main.latitude,geocodes.main.longitude,location.postcode
0,Bottega Fratelli,"[{'id': 13236, 'name': 'Italian Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}}]",18.480267,-69.932012,10122
1,Delacasa Pastelería,"[{'id': 13002, 'name': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}}]",18.478850,-69.934236,10122
2,Galería 360,"[{'id': 17114, 'name': 'Shopping Mall', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_', 'suffix': '.png'}}]",18.484544,-69.936048,10510
3,Ágora Mall,"[{'id': 17114, 'name': 'Shopping Mall', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_', 'suffix': '.png'}}]",18.483764,-69.939714,10122
4,"Licormart, Cash & Carry","[{'id': 17057, 'name': 'Food and Beverage Retail', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/foodanddrink_', 'suffix': '.png'}}]",18.483459,-69.934425,NaN


In [255]:
venues_df['categories'] = venues_df.apply(lambda x: x['categories'][0]['name'], axis=1)
venues_df.head()

,name,categories,geocodes.main.latitude,geocodes.main.longitude,location.postcode
0,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122
1,Delacasa Pastelería,Bakery,18.478850,-69.934236,10122
2,Galería 360,Shopping Mall,18.484544,-69.936048,10510
3,Ágora Mall,Shopping Mall,18.483764,-69.939714,10122
4,"Licormart, Cash & Carry",Food and Beverage Retail,18.483459,-69.934425,NaN


In [256]:
venues_df["location.postcode"] = venues_df["location.postcode"].fillna(0).astype(int)

In [257]:
venues_df = pd.merge(venues_df, 
                      df_data_0, 
                      left_on = 'location.postcode',
                      right_on = 'cod_postal' ,
                      how ='left')
del venues_df['cod_postal']
venues_df

,name,categories,geocodes.main.latitude,geocodes.main.longitude,location.postcode,Sectores
0,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco
1,Delacasa Pastelería,Bakery,18.478850,-69.934236,10122,Ensanche Naco
2,Galería 360,Shopping Mall,18.484544,-69.936048,10510,Viejo Arroyo Hondo
3,Ágora Mall,Shopping Mall,18.483764,-69.939714,10122,Ensanche Naco
4,"Licormart, Cash & Carry",Food and Beverage Retail,18.483459,-69.934425,0,NaN
5,Wendy’s,Fast Food Restaurant,18.475239,-69.926743,10122,Ensanche Naco
6,Food Truck Village,Food Truck,18.476136,-69.929394,0,NaN
7,Nipau,Mediterranean Restaurant,18.474405,-69.930105,10122,Ensanche Naco
8,Laurel Restaurant,Bistro,18.472318,-69.935076,0,NaN
9,Mijas Restaurante,Mediterranean Restaurant,18.474450,-69.933620,10150,Piantini


In [258]:
print('Existen {} categorias unicas.'.format(len(venues_df['categories'].unique())))
print('\n\nEstablecimientos en cada Sector: ')
venues_df.groupby('Sectores').count()

Existen 30 categorias unicas.


Establecimientos en cada Sector: 


,name,categories,geocodes.main.latitude,geocodes.main.longitude,location.postcode
Sectores,,,,,
El Millón,1,1,1,1,1
Ensanche Naco,15,15,15,15,15
La Esperilla,1,1,1,1,1
La Julia,1,1,1,1,1
Nuevo Arroyo Hondo,2,2,2,2,2
Paraíso,1,1,1,1,1
Piantini,13,13,13,13,13
Renacimiento,1,1,1,1,1
Viejo Arroyo Hondo,2,2,2,2,2


In [259]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='91LCVJBW8OyktK9Obt9l2hg1qxmaOvQHeErVDi4Nwzhs',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'holaproyectofinaldelcurso-donotdelete-pr-5yqimcvcp9bjmq'
object_key = 'Categorias_Act_Ec.xlsx'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']

df_data_1 = pd.read_excel(body.read())
df_data_1.head()


,categories,Actividad_Ec,Prom_Gastos,Prom_Utilidad,Prom_consumo,Prom_Rentabilidad
0,Italian Restaurant,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
1,Bakery,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
2,Shopping Mall,Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas,1.512540e+09,8.503943e+07,1.352839e+08,-0.249996
3,Shopping Mall,Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas,1.512540e+09,8.503943e+07,1.352839e+08,-0.249996
4,Food and Beverage Retail,Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas,1.512540e+09,8.503943e+07,1.352839e+08,-0.249996


In [260]:
venues_df = pd.merge(venues_df, 
                      df_data_1, 
                      on = 'categories',
                      how ='left')
venues_df

,name,categories,geocodes.main.latitude,geocodes.main.longitude,location.postcode,Sectores,Actividad_Ec,Prom_Gastos,Prom_Utilidad,Prom_consumo,Prom_Rentabilidad
0,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
1,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
2,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
3,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
4,Delacasa Pastelería,Bakery,18.478850,-69.934236,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
...,...,...,...,...,...,...,...,...,...,...,...
132,Wendy’s,Fast Food Restaurant,18.463924,-69.934080,10127,Piantini,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
133,Wendy’s,Fast Food Restaurant,18.463924,-69.934080,10127,Piantini,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
134,Wendy’s,Fast Food Restaurant,18.463924,-69.934080,10127,Piantini,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372
135,Wendy’s,Fast Food Restaurant,18.463924,-69.934080,10127,Piantini,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,-0.817372


In [261]:
downtown_onehot = pd.get_dummies(venues_df['categories'])

downtown_onehot['Sectores'] = venues_df['Sectores'] 

fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
print(downtown_onehot.shape)
downtown_onehot.head()

(137, 31)


,Sectores,Automotive Retail,Bakery,Bank,Bar,Bistro,Café,Cocktail Bar,Dessert Shop,Fast Food Restaurant,...,Middle Eastern Restaurant,Peruvian Restaurant,Pie Shop,Restaurant,Shopping Mall,Spanish Restaurant,Sushi Restaurant,Tennis Court,Tobacco Store,Wine Store
0,Ensanche Naco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ensanche Naco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ensanche Naco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Ensanche Naco,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ensanche Naco,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [262]:
downtown_grouped = downtown_onehot.groupby('Sectores').mean().reset_index()
print(downtown_grouped.shape)
downtown_grouped.head()

(9, 31)


,Sectores,Automotive Retail,Bakery,Bank,Bar,Bistro,Café,Cocktail Bar,Dessert Shop,Fast Food Restaurant,...,Middle Eastern Restaurant,Peruvian Restaurant,Pie Shop,Restaurant,Shopping Mall,Spanish Restaurant,Sushi Restaurant,Tennis Court,Tobacco Store,Wine Store
0,El Millón,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
1,Ensanche Naco,0.0,0.052632,0.0,0.0,0.0,0.052632,0.0,0.026316,0.078947,...,0.0,0.026316,0.0,0.105263,0.052632,0.0,0.026316,0.0,0.0,0.0
2,La Esperilla,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
3,La Julia,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.0,0.0
4,Nuevo Arroyo Hondo,0.0,0.000000,0.0,0.5,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [263]:
num_top_venues = 5

for hood in downtown_grouped['Sectores']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Sectores'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    
    temp = temp.iloc[1:]
    temp['freq'] = round(temp['freq'].astype(float),2)
    temp = temp.sort_values('freq', ascending=False).reset_index(drop=True)
    print(temp.head(num_top_venues))
    print('\n')

----El Millón----
               venue  freq
0  Café               1.0 
1  Automotive Retail  0.0 
2  Juice Bar          0.0 
3  Tobacco Store      0.0 
4  Tennis Court       0.0 


----Ensanche Naco----
                      venue  freq
0  Italian Restaurant        0.21
1  Mediterranean Restaurant  0.13
2  Restaurant                0.11
3  Fast Food Restaurant      0.08
4  Ice Cream Parlor          0.08


----La Esperilla----
                         venue  freq
0  Grocery Store / Supermarket  1.0 
1  Automotive Retail            0.0 
2  Juice Bar                    0.0 
3  Tobacco Store                0.0 
4  Tennis Court                 0.0 


----La Julia----
                venue  freq
0  Spanish Restaurant  1.0 
1  Automotive Retail   0.0 
2  Bakery              0.0 
3  Tobacco Store       0.0 
4  Tennis Court        0.0 


----Nuevo Arroyo Hondo----
                venue  freq
0  Bar                 0.5 
1  Mexican Restaurant  0.5 
2  Automotive Retail   0.0 
3  Juice Bar       

In [264]:
def return_most_common_venues(row, num_top_venues):
    row = row.iloc[1:]
    row_sorted = row.sort_values(ascending=False)
    
    return row_sorted.index.values[0:num_top_venues]

In [265]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Sectores']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Sectores'] = downtown_grouped['Sectores']

for ind in np.arange(downtown_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Sectores,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,El Millón,Café,Automotive Retail,Juice Bar,Tobacco Store,Tennis Court,Sushi Restaurant,Spanish Restaurant,Shopping Mall,Restaurant,Pie Shop
1,Ensanche Naco,Italian Restaurant,Mediterranean Restaurant,Restaurant,Fast Food Restaurant,Ice Cream Parlor,Grocery Store / Supermarket,Lounge,Bakery,Shopping Mall,Café
2,La Esperilla,Grocery Store / Supermarket,Automotive Retail,Juice Bar,Tobacco Store,Tennis Court,Sushi Restaurant,Spanish Restaurant,Shopping Mall,Restaurant,Pie Shop
3,La Julia,Spanish Restaurant,Automotive Retail,Bakery,Tobacco Store,Tennis Court,Sushi Restaurant,Shopping Mall,Restaurant,Pie Shop,Peruvian Restaurant
4,Nuevo Arroyo Hondo,Bar,Mexican Restaurant,Automotive Retail,Juice Bar,Tobacco Store,Tennis Court,Sushi Restaurant,Spanish Restaurant,Shopping Mall,Restaurant


In [266]:
k = 9

X = downtown_grouped.drop('Sectores', axis = 1)

kmeans = KMeans(n_clusters = k, random_state=0)
kmeans.fit(X)

KMeans(n_clusters=9, random_state=0)

In [267]:
venues_sorted['Cluster_Labels']=  kmeans.labels_

DN_merged = venues_df.join(venues_sorted.set_index('Sectores'), on='Sectores')

DN_merged.head() 

,name,categories,geocodes.main.latitude,geocodes.main.longitude,location.postcode,Sectores,Actividad_Ec,Prom_Gastos,Prom_Utilidad,Prom_consumo,...,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
0,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,...,Mediterranean Restaurant,Restaurant,Fast Food Restaurant,Ice Cream Parlor,Grocery Store / Supermarket,Lounge,Bakery,Shopping Mall,Café,4.0
1,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,...,Mediterranean Restaurant,Restaurant,Fast Food Restaurant,Ice Cream Parlor,Grocery Store / Supermarket,Lounge,Bakery,Shopping Mall,Café,4.0
2,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,...,Mediterranean Restaurant,Restaurant,Fast Food Restaurant,Ice Cream Parlor,Grocery Store / Supermarket,Lounge,Bakery,Shopping Mall,Café,4.0
3,Bottega Fratelli,Italian Restaurant,18.480267,-69.932012,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,...,Mediterranean Restaurant,Restaurant,Fast Food Restaurant,Ice Cream Parlor,Grocery Store / Supermarket,Lounge,Bakery,Shopping Mall,Café,4.0
4,Delacasa Pastelería,Bakery,18.478850,-69.934236,10122,Ensanche Naco,Actividades de alojamiento y de servicio de comidas,3.754133e+08,-1.750524e+08,2.417025e+08,...,Mediterranean Restaurant,Restaurant,Fast Food Restaurant,Ice Cream Parlor,Grocery Store / Supermarket,Lounge,Bakery,Shopping Mall,Café,4.0


In [268]:
DN_merged['Cluster_Labels'] =DN_merged['Cluster_Labels'].fillna(0).astype(int)

In [269]:
map_clusterd = folium.Map(location=[18.483402,-69.929611], zoom_start=14)



x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(DN_merged['geocodes.main.latitude'], DN_merged['geocodes.main.longitude'],
                                  DN_merged['Sectores'], DN_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusterd)
    
map_clusterd

# Resultados 
El objetivo de este proyecto es poder ayudar a las personas a emprender un negocio en el Distrito Nacional, Santo Domingo, República Dominicana, basándonos en su ubicación y tipo de negocio, es decir, su actividad económica. Con los resultados obtenidos las personas se pudieran interesar en localizarse en el sector con una mayor cantidad de negocios, en este caso la mejor opción sería el clúster 4 en donde el negocio que ser repite más frecuente mente son los restaurantes italianos. Si el interés es abrir un restaurante, otra buena opción es el clúster 8 donde el negocio más común son los restaurantes mediterráneos. Los negocios de restaurantes caen dentro de la actividad económica “Actividades de alojamiento y de servicio de comidas” él es un negocio que conlleva en promedio menores gastos recurrentes que los otros negocios más comunes de estos sectores. Por otro lado, si el interés es abrir algún negocio comercial, que, aunque su sector económico conlleve mayores gastos tienen una rentabilidad promedio más alta que los otros negocios, lo ideal es iniciar con este negocio en los clústeres 2 y 7, ya que los negocios que se repiten con más frecuencia alrededor de estos sectores pertenecen a la actividad económica de comercio.

# Conclusión 
Este proyecto ayuda a las personas a comprender mejor los sectores del Distrito Nacional, Santo Domingo respecto a los lugares más comunes de ese Sector. Siempre es muy importante hacer uso de los datos antes de tomar una decisión tan importante como la de abrir un negocio, tanto para investigar el sector ideal en donde abrir ese negocio como qué tipo de negocio sería ideal para emprender. El futuro de este proyecto pretende incluir otros factores como los niveles de seguridad de cada sector, para poder seleccionar el sector ideal en función de los negocios más comunes, los gastos recurrentes y  la seguridad de cada sector. 
